In [62]:
import pandas as pd
import tweepy
import requests
import os
import json
import time

## Gather data

Read in provided data to a dataframe.

In [63]:
ea = pd.read_csv('twitter-archive-enhanced.csv') # Read in enhanced twitter archive

Now use the requests library to programmatically download the `image-predictions.tsv` file from 

In [64]:
r = requests.get(
    'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

Write image predictions data in to a `.tsv` file.

In [65]:
with open('image-predictions.tsv', 'w', encoding='utf-8') as file: 
    file.write(str(r.text))

Read `.tsv` file in to dataframe `ip`.

In [66]:
ip = pd.read_csv('image-predictions.tsv', sep = '\t')

In [67]:
ip.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [68]:
ea.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available fo...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know w...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD3...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and sh...",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/st...",12,10,Franklin,None,None,None,None


Grab all the tweet ids.

In [69]:
ids = ea['tweet_id']

Cycle through all the tweet ids, and grab their JSON data from the Twitter API using `tweepy` and `json`.

Many tweets have been deleted, and so cause an error to be thrown.

In [70]:
import tweepy

consumer_key = '2VDcRqXbcihwqtob3KyCHR4HA'
consumer_secret = 'Q2eRUisXRiu5mpFgakTyUQwpiIAGy4xHRC9fNW8I0FFCckGCdE'
access_token = '773745310661275652-rAdthEP5lVG6yiF4o2OydJnd3WGWryV'
access_secret = 'QbAWTTc2yah7G8GOtFQxAgGKH9iJT0liT1qy3zPvqhT8N'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True,
                wait_on_rate_limit_notify = True)

Grab all the JSON data from tweepy and store it in `tweet_json.txt`. If an error is thrown when we try to access a tweet this probably means the tweet was deleted. The `try` `except` block allows us to continue on despite errors.

with open('tweet_json.txt', mode = 'w', encoding='utf-8') as file:
    for x in ids:
        try:
            tweet = api.get_status(x, tweet_mode='extended')
            json.dump(tweet._json, file)
            print(x)
        except:
            print('This tweet ID: {0}, raised an error.'.format(x))

The JSON data is not parseable by the `json` decoder. [This stackoverflow answer](https://stackoverflow.com/questions/21708192/how-do-i-use-the-json-module-to-read-in-one-json-object-at-a-time/21709058#21709058) proved very useful parsing the following JSON data in `tweet_json.txt`. 

In [71]:
from functools import partial

def json_parse(fileobj, decoder=json.JSONDecoder(), buffersize=2048):
    buffer = ''
    for chunk in iter(partial(fileobj.read, buffersize), ''):
        buffer += chunk
        while buffer:
            try:
                result, index = decoder.raw_decode(buffer)
                yield result
                buffer = buffer[index:]
            except ValueError:
                break

Build a list of dictionaries using `json_parse()` to read the JSON objects individually, then create a new dataframe with our new information from the Twitter api.

In [72]:
df_list = []

with open('tweet_json.txt', mode='r', encoding='utf-8') as file:
    for data in json_parse(file):
        df_list.append({'rt_count': data['retweet_count'],
                        'fav_count': data['favorite_count'],
                        'tweet_id': data['id']})

In [73]:
cols = ['rt_count', 'fav_count', 'tweet_id']

from_api = pd.DataFrame(df_list,
                        columns = cols)

## Assessment

In [74]:
df_clean = ea.copy()

In [75]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [76]:
ip.head(20)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [77]:
ip.img_num.describe()

count    2075.000000
mean        1.203855
std         0.561875
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         4.000000
Name: img_num, dtype: float64

In [78]:
ip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [79]:
df_clean[df_clean.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [80]:
ip[ip.duplicated()]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


In [81]:
ip['p1_conf'].describe()

count    2075.000000
mean        0.594548
std         0.271174
min         0.044333
25%         0.364412
50%         0.588230
75%         0.843855
max         1.000000
Name: p1_conf, dtype: float64

In [82]:
ip['p2_conf'].describe()

count    2.075000e+03
mean     1.345886e-01
std      1.006657e-01
min      1.011300e-08
25%      5.388625e-02
50%      1.181810e-01
75%      1.955655e-01
max      4.880140e-01
Name: p2_conf, dtype: float64

In [83]:
ip['p3_conf'].describe()

count    2.075000e+03
mean     6.032417e-02
std      5.090593e-02
min      1.740170e-10
25%      1.622240e-02
50%      4.944380e-02
75%      9.180755e-02
max      2.734190e-01
Name: p3_conf, dtype: float64

In [84]:
df_clean.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


## Quality issues

### Quality issues with the tweets dataframe

* Datatype issues:
 1. `tweet_id` should be a string not an int.
 2. `timestamp` should be a datetime object
* The `source` column has html tags still around it.
* All data that's a retweet should be dropped.
* All data that's a reply should be dropped.
* The `in_reply_to...` columns should be dropped. They're almost entirely `NaN`s. The same can be said for the `retweeted_status...` columns.
* Not all tweets have an image in them

### Quality issues with the image predictions dataframe:
* `tweet_id` should be a string not an int
* `p1`, `p2`, and `p3` columns have '\_' instead of spaces. There's also inconsistency in their capitalization.

### Api data issues
* `rt_count` and `fav_count` should be ints rather than floats.

## Tidiness issues

* The api dataframe and the enhanced tweets dataframe both have data on the same set of tweets, meaning each type of observational unit does *not* form a table.
* `doggo`, `floofer`, `pupper`, and `puppo` are not variables, but data. These shouldn't be in the columns.

## Cleaning

#### *__`tweet_id` should be a string__*.

We'll fix this by using `.astype()`.

In [85]:
df_clean['tweet_id'] = df_clean['tweet_id'].astype(str)

Verify that the data type is a string:

In [86]:
df_clean['tweet_id'].dtype

dtype('O')

---------

#### *__`timestamp`__* should be a datetime object.

We'll fix this by using `pd.to_datetime`.

In [87]:
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])

Verify that the column is `datetime` type. This should show up as `<M8[ns]`.

In [88]:
df_clean['timestamp'].dtype

dtype('<M8[ns]')

---------------------

#### The `source` column still has html tags around it.

We grab everything inside the tag.

In [89]:
# We'll use regex to solve this problem
import re

In [90]:
# Let's us see all the contents of 'source'
pd.set_option('max_colwidth', 100)

Test and confirm our regex pattern works:

In [91]:
df_clean.source.head(100)

0     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
1     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
2     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
3     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
4     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
5     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
6     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
7     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
8     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
9     <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
10    <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>
11    <a href="http:/

In [92]:
pattern = re.compile(r'<a.*>([\w\s]+)</a>')

In [93]:
match = pattern.search('<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>')

In [94]:
match.groups()

('Twitter for iPhone',)

In [95]:
df_clean.source = df_clean.source.str.extract(pat = pattern)

C:\Users\Luke\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


Test to see if we've successfully gotten rid of the html tags:

In [96]:
df_clean.source.head()

0    Twitter for iPhone
1    Twitter for iPhone
2    Twitter for iPhone
3    Twitter for iPhone
4    Twitter for iPhone
Name: source, dtype: object

---------------------- 

#### Data from retweets should be removed.

Rows with non-null values in  `Retweeted_status...` columns should be dropped, as these are retweets.

In [97]:
df_clean[['retweeted_status_id', 'retweeted_status_user_id',
         'retweeted_status_timestamp']].notnull().sum()

retweeted_status_id           181
retweeted_status_user_id      181
retweeted_status_timestamp    181
dtype: int64

All three columns share the same rows with non-null values, so we'll drop the rows where `retweeted_status_id` is not null (it doesn't matter which column we choose).

In [98]:
df_clean = df_clean[df_clean['retweeted_status_id'].isnull()]

Now let's check the `in_reply_to...` columns to see if they have meet the same criteria as `retweeted_status...` columns:

In [99]:
df_clean[df_clean['in_reply_to_status_id'].notnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35,Twitter for iPhone,@NonWhiteHat @MayhewMayhem omg hello tanner you are a scary good boy 12/10 would pet with extrem...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53,Twitter for iPhone,@roushfenway These are good dogs but 17/10 is an emotional impulse rating. More like 13/10s,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36,Twitter for iPhone,@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25,Twitter for iPhone,@ComplicitOwl @ShopWeRateDogs &gt;10/10 is reserved for dogs,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35,Twitter for iPhone,"@Jack_Septic_Eye I'd need a few more pics to polish a full analysis, but based on the good boy c...",NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
149,863079547188785154,6.671522e+17,4.196984e+09,2017-05-12 17:12:53,Twitter for iPhone,"Ladies and gentlemen... I found Pipsy. He may have changed his name to Pablo, but he never chang...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/863079547188785154/photo/1,14,10,None,None,None,None,None
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51,Twitter for iPhone,@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
184,856526610513747968,8.558181e+17,4.196984e+09,2017-04-24 15:13:52,Twitter for iPhone,"THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY HI AFTER ALL. PUPGRADED TO A 14/10. WOULD BE AN H...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/856526610513747968/photo/1,14,10,None,None,None,None,None
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03,Twitter for iPhone,@xianmcguire @Jenna_Marbles Kardashians wouldn't be famous if as a society we didn't place enorm...,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32,Twitter for iPhone,@dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None


A visual inspection of these data shows they aren't retweets. However, they represent a very small part of the dataset -- I'm going to drop rows where these columns aren't null in the interest of consistency.

After dropping these rows, our dataset will be purely tweets. No retweets, and no replies.

In [100]:
df_clean = df_clean[df_clean['in_reply_to_status_id'].isnull()]

#### __The `in_reply_to...` and `retweet_status...` columns should be dropped.

We'll verify that these columns are all NaN's and then drop them.

In [101]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2097 non-null object
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     2097 non-null datetime64[ns]
source                        2006 non-null object
text                          2097 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2094 non-null object
rating_numerator              2097 non-null int64
rating_denominator            2097 non-null int64
name                          2097 non-null object
doggo                         2097 non-null object
floofer                       2097 non-null object
pupper                        2097 non-null object
puppo                         2097 non-null object
dtypes: datetime64[ns](1), floa

In [102]:
to_drop = ['in_reply_to_status_id', 'in_reply_to_user_id',
           'retweeted_status_id', 'retweeted_status_user_id',
           'retweeted_status_timestamp']

df_clean = df_clean.drop(to_drop, axis=1)

--------

#### __ Not all tweets have an image in them.__
Use pandas `.str.extract` with a regular expression to extract all tweets whose urls have the word "photo" in them.

In [103]:
df_clean = df_clean.loc[df_clean['expanded_urls'].str.extract(pat=r'(photo)').notnull(), :]

C:\Users\Luke\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [112]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1899 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              1899 non-null object
timestamp             1899 non-null datetime64[ns]
source                1899 non-null object
text                  1899 non-null object
expanded_urls         1899 non-null object
rating_numerator      1899 non-null int64
rating_denominator    1899 non-null int64
name                  1899 non-null object
doggo                 1899 non-null object
floofer               1899 non-null object
pupper                1899 non-null object
puppo                 1899 non-null object
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 192.9+ KB


-------------


## Image predictions dataframe cleaning

First make a copy of image predictions dataframe.

In [105]:
ip_clean = ip.copy()

#### __`tweet_id`__  should be a string not an int.

In [106]:
ip_clean.tweet_id = ip_clean.tweet_id.astype(str)

Check datatype to ensure `tweet_id` is now a string:

In [107]:
ip_clean.tweet_id.dtype

dtype('O')

-------------

#### __`p1`, `p2`, and `p3` columns have '\_' instead of spaces. There's also inconsistency in their capitalization.__

According to [this](https://style.mla.org/capitalizing-names-of-dog-breeds/) capitalization guide, most dog names only have the first word capitalized. We will follow this convention.

In [108]:
ip_clean['p1'].str.replace('_', ' ').str.capitalize()

0            Welsh springer spaniel
1                           Redbone
2                   German shepherd
3               Rhodesian ridgeback
4                Miniature pinscher
5              Bernese mountain dog
6                        Box turtle
7                              Chow
8                     Shopping cart
9                  Miniature poodle
10                 Golden retriever
11                    Gordon setter
12                     Walker hound
13                              Pug
14                       Bloodhound
15                            Lhasa
16                   English setter
17                              Hen
18                 Desktop computer
19                Italian greyhound
20                      Maltese dog
21                 Three-toed sloth
22                               Ox
23                 Golden retriever
24                         Malamute
25                       Guinea pig
26      Soft-coated wheaten terrier
27                        Ch

In [109]:
to_fix = ['p1', 'p2', 'p3']

for col in to_fix:
    ip_clean[col] = ip_clean[col].str.replace('_', ' ').str.capitalize()

Let's check to see the desired changes have been made:

In [110]:
ip_clean[['p1', 'p2', 'p3']]

,p1,p2,p3
0,Welsh springer spaniel,Collie,Shetland sheepdog
1,Redbone,Miniature pinscher,Rhodesian ridgeback
2,German shepherd,Malinois,Bloodhound
3,Rhodesian ridgeback,Redbone,Miniature pinscher
4,Miniature pinscher,Rottweiler,Doberman
5,Bernese mountain dog,English springer,Greater swiss mountain dog
6,Box turtle,Mud turtle,Terrapin
7,Chow,Tibetan mastiff,Fur coat
8,Shopping cart,Shopping basket,Golden retriever
9,Miniature poodle,Komondor,Soft-coated wheaten terrier


Merge the enhanced archive that was provided, the newly obtained data from the Twitter api, and the the enhanced predictions to create our new dataframe with all the info we want.

In [111]:
df = pd.merge(ea, from_api, how='outer', on='tweet_id')